In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# URL da página da Wikipedia
url = 'https://pt.wikipedia.org/wiki/Lista_dos_maiores_est%C3%A1dios_de_futebol_do_Brasil'
url_base = 'https://pt.wikipedia.org'

# Enviar uma solicitação GET para a página
response = requests.get(url)

# Verificar se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Analisar o conteúdo da página com BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrar a minha tabela desejada
    my_table = soup.find("table", {"class": "wikitable"})
    
    # Encontrar todos os links dentro da tabela
    stadium_links = my_table.find_all('a', href=True)

    # Filtrar apenas os links que têm "Estádio" no atributo 'title'
    stadium_links = [link for link in stadium_links if 'Estádio' in link.get('title', '')]
    
    # Iterar pelos links e imprimir seus URLs
    complete_stadium_links = []
    for link in stadium_links:
        if 'página não existe' in link.get('title', ''):
            pass
        else:
            complete_link = url_base + link['href']
            complete_stadium_links.append(complete_link)
        
else:
    print(f'Não foi possível acessar a página. Código de status: {response.status_code}')

In [ ]:
complete_stadium_links[:10]

In [ ]:



stadium_names = []
stadium_details = []

for link in complete_stadium_links:

    # Enviar uma solicitação GET para a página
    stadium_page = requests.get(link)

    # Verificar se a solicitação foi bem-sucedida
    if stadium_page.status_code == 200:
        try:
            # Analisar o conteúdo da página com BeautifulSoup
            page = BeautifulSoup(stadium_page.text, 'html.parser')

            # Encontrar a minha tabela desejada
            my_table = page.find("table", {"class": "infobox infobox_v2"})
            
            # Encontrar o nome do estadio
            nome_estadio = my_table.find('th').get_text(strip=True)
            stadium_names.append(nome_estadio) # Insere o nome do estadio na lista de nomes de estadio
            print(nome_estadio)
            print("------------------------------------")
            
            # Lista com as linhas desejadas
            rows = [] # cria uma lista vazia

            wanted_values = ['nome', 'apelido', 'data', 'público recorde', 'data recorde', 'partida com mais público']

            link_atual = ['link', link]
            rows.append(link_atual)
            
            for row in my_table.find_all('tr')[1:]: 
                value = row.find_all('td')  
                                    
                beautified_value = [ele.text.strip() for ele in value] 
                
                if len(beautified_value) == 0:                         
                    continue
            
                if str.lower(beautified_value[0]) in wanted_values:
                    rows.append(beautified_value) # Insere as linhas desejadas na lista 'rows'       
        # fim try
        
        except Exception as error:
            continue
    # fim if

    else:
        print(f'Não foi possível acessar a página. Código de status: {response.status_code}')

    stadium_details.append(rows)
    

In [ ]:
stadium_details_dict = {}

# Transforma as listas com os nomes dos estadios e os detalhes em um dicionario 
for i in range(len(stadium_names)):
    key = stadium_names[i]
    value = stadium_details[i]
    stadium_details_dict[key] = value

# Iterar pelo dicionário
for chave, valor in stadium_details_dict.items():
    print(chave)                                 # Imprime a chave
    for item in valor:                           # Itera pelas listas internas
        print(item)                              # Imprime cada lista interna
    print()
                                  # Quebra de linha entre as entradas do dicionário


In [ ]:
for stadium, info in stadium_details_dict.items():
    stadium_info = {}

    for item in info:
        stadium_info[item[0]] = item[1]
        
    stadium_details_dict[stadium] = stadium_info

In [ ]:
# Iterar pelo dicionário
for chave, valor in stadium_details_dict.items():
    print(chave)                                               # Imprime a chave
    for item, info in valor.items():                           # Itera pelas listas internas
        print(item, ": ", info)                                # Imprime cada lista interna
    print()
    

In [ ]:
stadium_details_dict['Maracanã']

In [ ]:
df = pd.DataFrame.from_dict(stadium_details_dict, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'Estadio',
                   'Nome': 'Nome Oficial' ,
                   'Data': 'Data Inauguracao'}, inplace=True)
df


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df['Estadio'] = df['Estadio'].str.replace('Estádio', ' Estádio')
df.head()

In [ ]:
df.to_csv('brazilian-stadiums-details.csv', index=False)

oq eu quero?

* Nome Oficial
* Apelido
* data construção
* data inauguração
* Publico recorde
* data Recorde
* partida com mais publico
